In [1]:
import pysam

pysam.index("./data/merged-tumor.bam")
bam_file = pysam.AlignmentFile("./data/merged-tumor.bam", "rb")

In [4]:
for read in bam_file:
    print(read)
    break

C0HVYACXX120402:7:1207:5722:57044	1187	#20	9483249	27	76M	#20	9483382	209	TTTTCAAACAGTATCTATGCCTGCCAAATGTGAACATATAAAAAAAAACCAGAATGTGCCATTCTGATTTAAACTG	array('B', [28, 28, 27, 29, 31, 30, 31, 31, 29, 31, 35, 30, 29, 31, 34, 30, 29, 23, 41, 32, 20, 30, 29, 34, 34, 29, 30, 31, 30, 30, 30, 33, 33, 26, 39, 12, 25, 19, 32, 30, 35, 28, 35, 33, 23, 33, 35, 36, 30, 38, 33, 41, 34, 35, 31, 33, 23, 30, 30, 36, 27, 32, 29, 34, 35, 41, 33, 31, 33, 29, 32, 32, 31, 31, 31, 34])	[('XA', 'GL000217.1,-110754,76M,1;'), ('BD', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('MD', '76'), ('RG', '1'), ('BI', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('NM', 0), ('MQ', 27), ('AS', 76), ('XS', 71)]


In one read there are the following informations separated by tab:
1. **QueryName (QNAME)** - contains name of the read
2. **Flags (FLAG)** - integer value that contains bitwise flags summarizing various properties of the read alignment
3. **Reference Name (RNAME)** - name of the reference sequence to which the read has been aligned
4. **Position (POS)** - 1-based leftmost mapping position of the first CIGAR operation that “consumes” a reference base
5. **Mapping Quality (MAPQ)** - mapping quality of read
6. **CIGAR string (CIGAR)** - describes alignment details in terms of operations like match, mismatch, insertion, deletion, and length of those operations
7. **Mate Reference Name (MRNM)** - contains reference name of the mate/next read
8. **Mate Position (MPOS)** - represents 1-based mapping position of the mate/next read
9. **Template Length (TLEN)** - represents signed observed template length
10. **Segment Sequence (SEQ)** - represents the actual sequence of nucleotides in the read. It is the sequence of bases that were sequenced by the sequencing machine. It is usually represented as a string of letters corresponding to the nucleotides (A, C, G, T, and sometimes N for ambiguous bases)
11. **ASCII of base quality (QUAL)** - represents the base quality scores associated with each base in the sequence. Base quality scores indicate the confidence level of each base call. They are typically represented as ASCII characters, with each character corresponding to a numerical value representing the Phred quality score. These scores are usually generated by the sequencing machine during the sequencing process. A higher quality score indicates a higher confidence in the base call.
12. **Some of alignment tags (Optional fields):**
    - XA - alternative mapping positions for the read
    - BD - per-base attributes of the aligned read
    - MD - describes the mismatches and deletions in the read alignment compared to the reference
    - RG - indicates the read group to which the read belongs
    - BI - provides the per-base alignment index
    - NM - indicates the number of mismatches in the alignment
    - MQ - represents the mapping quality of the read
    - AS - represents the alignment score
    - XS - represents the alignment score for the best alignment found

How many unmapped reads are in the file?

In [5]:
# samtools view -c -f 4 merged-tumor.bam
# output: 17765

unmapped_reads = [read for read in bam_file.fetch(until_eof=True) if read.is_unmapped]

print("Number of unmapped reads is", len(unmapped_reads))

Number of unmapped reads is 17765


Calculate total number of reads

In [6]:
# samtools view -c merged-tumor.bam
# output: 2921629

num_reads = bam_file.count()

print("Total number of reads is", num_reads)

Total number of reads is 2921629


Calculate number of reads with mapping quality 0

In [7]:
# samtools view -c merged-tumor.bam
# output: 2921629
# samtools view -c -q 1 merged-tumor.bam
# output: 2795001
# 2921629 - 2795001 = 126628

num_of_zero_mapping_quality_reads = sum(1 for read in bam_file.fetch() if read.mapping_quality == 0)

print("Number of reads with mapping quality 0 is", num_of_zero_mapping_quality_reads)

Number of reads with mapping quality 0 is 126628


Calculate average mapping quality for all the reads

In [8]:
total_mapping_quality = sum(read.mapping_quality for read in bam_file.fetch())

average_mapping_quality = total_mapping_quality / num_reads if num_reads > 0 else 0

print("Average mapping quality for all reads is", average_mapping_quality)

Average mapping quality for all reads is 55.91379158681681


Calculate average mapping quality if reads with 0 mapping quality are filtered out

In [9]:
total_mapping_quality = 0
num_reads = 0

for read in bam_file.fetch():
    if read.mapping_quality > 0:
        total_mapping_quality += read.mapping_quality
        num_reads += 1

average_mapping_quality = total_mapping_quality / num_reads if num_reads > 0 else 0

bam_file.close()

print("Average mapping quality for all reads is", average_mapping_quality)

Average mapping quality for all reads is 58.446975510921106
